<a href="https://colab.research.google.com/github/chiamaka249/IgboNER/blob/main/Copy_of_igbobert5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -c https://github.com/IgnatiusEzeani/IGBONLP/raw/master/ig_monoling/text.zip
!wget -c https://raw.githubusercontent.com/chiamaka249/lacuna_pos_ner/main/language_corpus/ibo/ibo.txt
# !wget -c https://github.com/chiamaka249/IgboNER/blob/main/config.json

--2021-12-10 13:55:37--  https://github.com/IgnatiusEzeani/IGBONLP/raw/master/ig_monoling/text.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IgnatiusEzeani/IGBONLP/master/ig_monoling/text.zip [following]
--2021-12-10 13:55:37--  https://raw.githubusercontent.com/IgnatiusEzeani/IGBONLP/master/ig_monoling/text.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7604282 (7.3M) [application/zip]
Saving to: ‘text.zip’

text.zip            100%[===================>]   7.25M  --.-KB/s    in 0.08s   

2021-12-10 13:55:37 (86.9 MB/s) - ‘text.zip’ saved [7604282/7604282]

--2021-12-10 13:55:37--  ht

In [ ]:
import zipfile
import os


def unzip(zipfilename):
  try:
    with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
      zip_ref.extractall(zipfilename[:-4])
      return f"'{zipfilename}' unzipped!"
  except FileNotFoundError:
    print(f"Cannot find '{zipfilename}' file")

unzip("text.zip")
!rm text.zip

In [ ]:
#copies the file "ibo.txt" to into the folder "text"
import shutil
shutil.copy('/content/ibo.txt', '/content/text')



'/content/text/ibo.txt'

In [ ]:
# import os
#import shutil
dir_name = "/content/text"
text=""
for fname in os.listdir(dir_name):
  fname = os.path.join(dir_name, fname)
  with open(fname, "r", encoding="utf8") as datafile:
    text = text+"\n"+datafile.read()

with open("data.txt", "w", encoding="utf8") as datafile:
  datafile.write(text)

shutil.rmtree("text")

In [ ]:
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1

Found existing installation: tensorflow 2.7.0
Uninstalling tensorflow-2.7.0:
  Successfully uninstalled tensorflow-2.7.0
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-7hq954v8
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-7hq954v8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 61 kB 435 kB/s 
     |████████████████████████████████| 3.3 MB 10.8 MB/s 
     |████████████████████████████████| 596 kB 47.7 MB/s 
     |████████████████████████████████| 895 kB 58.0 MB/s 
  Created wheel for transformers: filename=transformers-4.14.0.dev0-py3-none-any.whl size=3320694 sha256=ea466d0c77aee645aff42d3c4687f0ad0ca7ee78eebe49164f31b39d9d9ca629
  Stored in directory: /tmp/pip-ephem-wheel-cache-h4tuxkfa/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attem

In [ ]:
%%time 
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer

paths = [str(x) for x in Path(".").glob("**/*.txt")]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

CPU times: user 24.8 s, sys: 1.74 s, total: 26.5 s
Wall time: 7.34 s


In [ ]:
!mkdir igbo_bert4
tokenizer.save_model("igbo_bert4")

['igbo_bert4/vocab.json', 'igbo_bert4/merges.txt']

In [ ]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

tokenizer = ByteLevelBPETokenizer(
    "./igbo_bert4/vocab.json",
    "./igbo_bert4/merges.txt",
)

In [ ]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [ ]:
tokenizer.encode("Simone gara ụka ụnyahụ guọ egwu ma ga-kwa taa.").tokens

['<s>',
 'Simone',
 'Ġgara',
 'Ġá»¥ka',
 'Ġá»¥nyahá»¥',
 'Ġgu',
 'á»į',
 'Ġegwu',
 'Ġma',
 'Ġga',
 '-',
 'kwa',
 'Ġtaa',
 '.',
 '</s>']

In [ ]:
# Check that we have a GPU
!nvidia-smi

Fri Dec 10 14:01:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [ ]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [ ]:
#from google.colab import files
#files.upload()

In [ ]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("./igbo_bert4", max_len=512)

In [ ]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [ ]:
model.num_parameters()
# => 83 million parameters

83504416

In [ ]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer = tokenizer,
    file_path = "/content/data.txt",
    block_size = 128
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


CPU times: user 42.2 s, sys: 1.54 s, total: 43.8 s
Wall time: 19.3 s


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./igbo_bert4",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [ ]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
***** Running training *****
  Num examples = 391448
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 30585
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss
500,6.385000
1000,5.254600
1500,4.849400
2000,4.510000
2500,4.272500
3000,4.124000
3500,3.952800
4000,3.885700
4500,3.721500
5000,3.662000


Saving model checkpoint to ./igbo_bert4/checkpoint-10000
Configuration saved in ./igbo_bert4/checkpoint-10000/config.json
Model weights saved in ./igbo_bert4/checkpoint-10000/pytorch_model.bin
Saving model checkpoint to ./igbo_bert4/checkpoint-20000
Configuration saved in ./igbo_bert4/checkpoint-20000/config.json
Model weights saved in ./igbo_bert4/checkpoint-20000/pytorch_model.bin
Saving model checkpoint to ./igbo_bert4/checkpoint-30000
Configuration saved in ./igbo_bert4/checkpoint-30000/config.json
Model weights saved in ./igbo_bert4/checkpoint-30000/pytorch_model.bin
Deleting older checkpoint [igbo_bert4/checkpoint-10000] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)




CPU times: user 3h 9min 10s, sys: 3min 21s, total: 3h 12min 31s
Wall time: 3h 11min 17s


TrainOutput(global_step=30585, training_loss=3.1476998701309173, metrics={'train_runtime': 11477.3182, 'train_samples_per_second': 170.531, 'train_steps_per_second': 2.665, 'total_flos': 3.357401044877568e+16, 'train_loss': 3.1476998701309173, 'epoch': 5.0})

In [ ]:
trainer.save_model("./igbo_bert4")

Saving model checkpoint to ./igbo_bert4
Configuration saved in ./igbo_bert4/config.json
Model weights saved in ./igbo_bert4/pytorch_model.bin


In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./igbo_bert4",
    tokenizer="./igbo_bert4"
)

loading configuration file ./igbo_bert4/config.json
Model config RobertaConfig {
  "_name_or_path": "./igbo_bert4",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.14.0.dev0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 52000
}

loading configuration file ./igbo_bert4/config.json
Model config RobertaConfig {
  "_name_or_path": "./igbo_bert4",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dro

In [ ]:
# The sun <mask>.
# =>

fill_mask("Abụ m Maazị <mask>.") #= okafor/Ọkafọ
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.010340388864278793,
  'sequence': 'Abụ m Maazị A.',
  'token': 347,
  'token_str': ' A'},
 {'score': 0.009177258238196373,
  'sequence': 'Abụ m Maazị O.',
  'token': 377,
  'token_str': ' O'},
 {'score': 0.0053979880176484585,
  'sequence': 'Abụ m Maazị John.',
  'token': 2772,
  'token_str': ' John'},
 {'score': 0.005330500192940235,
  'sequence': 'Abụ m Maazị Ọkafọ.',
  'token': 5576,
  'token_str': ' Ọkafọ'},
 {'score': 0.005150600336492062,
  'sequence': 'Abụ m Maazị Nwaanyị.',
  'token': 2822,
  'token_str': ' Nwaanyị'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask("Nwaanyị na <mask> ji na akara.") #= eri
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.1287631392478943,
  'sequence': 'Nwaanyị na- ji na akara.',
  'token': 17,
  'token_str': '-'},
 {'score': 0.11695194989442825,
  'sequence': 'Nwaanyị na ya ji na akara.',
  'token': 290,
  'token_str': ' ya'},
 {'score': 0.10842421650886536,
  'sequence': 'Nwaanyị na nwunye ji na akara.',
  'token': 732,
  'token_str': ' nwunye'},
 {'score': 0.036929309368133545,
  'sequence': 'Nwaanyị na nwaanyị ji na akara.',
  'token': 620,
  'token_str': ' nwaanyị'},
 {'score': 0.019814301282167435,
  'sequence': 'Nwaanyị na nna ji na akara.',
  'token': 717,
  'token_str': ' nna'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask("Chineke ga- ebibikwa ndị niile na- eme ihe <mask>.") #=ọjọọ
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.29081615805625916,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe ọjọọ.',
  'token': 718,
  'token_str': ' ọjọọ'},
 {'score': 0.13454636931419373,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe ọma.',
  'token': 498,
  'token_str': ' ọma'},
 {'score': 0.06475759297609329,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe niile.',
  'token': 430,
  'token_str': ' niile'},
 {'score': 0.04965321347117424,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe anya.',
  'token': 402,
  'token_str': ' anya'},
 {'score': 0.04515065252780914,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe ike.',
  'token': 362,
  'token_str': ' ike'}]

In [ ]:
fill_mask("ọba akwụkwọ Ọkammụta Kenneth Dike dị <mask>.") #n'Awka

# This is the beginning of a beautiful <mask>.
# =>

[{'score': 0.23585939407348633,
  'sequence': 'ọba akwụkwọ Ọkammụta Kenneth Dike dị iche.',
  'token': 442,
  'token_str': ' iche'},
 {'score': 0.06350675970315933,
  'sequence': 'ọba akwụkwọ Ọkammụta Kenneth Dike dị icheiche.',
  'token': 3762,
  'token_str': ' icheiche'},
 {'score': 0.061406027525663376,
  'sequence': 'ọba akwụkwọ Ọkammụta Kenneth Dike dị mkpa.',
  'token': 614,
  'token_str': ' mkpa'},
 {'score': 0.04727039113640785,
  'sequence': 'ọba akwụkwọ Ọkammụta Kenneth Dike dị ukwuu.',
  'token': 1040,
  'token_str': ' ukwuu'},
 {'score': 0.035409118980169296,
  'sequence': 'ọba akwụkwọ Ọkammụta Kenneth Dike dị ndụ.',
  'token': 534,
  'token_str': ' ndụ'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask("Nwaanyị na eri <mask> na akara.") #= ji
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.04619939625263214,
  'sequence': 'Nwaanyị na eri nri na akara.',
  'token': 873,
  'token_str': ' nri'},
 {'score': 0.04371406137943268,
  'sequence': 'Nwaanyị na eri ya na akara.',
  'token': 290,
  'token_str': ' ya'},
 {'score': 0.034361857920885086,
  'sequence': 'Nwaanyị na eri ihe na akara.',
  'token': 302,
  'token_str': ' ihe'},
 {'score': 0.02798701822757721,
  'sequence': 'Nwaanyị na eri ego na akara.',
  'token': 590,
  'token_str': ' ego'},
 {'score': 0.025064406916499138,
  'sequence': 'Nwaanyị na eri nwaanyị na akara.',
  'token': 620,
  'token_str': ' nwaanyị'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask("Gaanụ mee ndị <mask> niile ka ha bụrụ ndị na- eso ụzọ m  .") #= mba


[{'score': 0.4334820806980133,
  'sequence': 'Gaanụ mee ndị a niile ka ha bụrụ ndị na- eso ụzọ m .',
  'token': 266,
  'token_str': ' a'},
 {'score': 0.057006802409887314,
  'sequence': 'Gaanụ mee ndị mmadụ niile ka ha bụrụ ndị na- eso ụzọ m .',
  'token': 389,
  'token_str': ' mmadụ'},
 {'score': 0.054193347692489624,
  'sequence': 'Gaanụ mee ndị ọzọ niile ka ha bụrụ ndị na- eso ụzọ m .',
  'token': 434,
  'token_str': ' ọzọ'},
 {'score': 0.05103715509176254,
  'sequence': 'Gaanụ mee ndị okenye niile ka ha bụrụ ndị na- eso ụzọ m .',
  'token': 1028,
  'token_str': ' okenye'},
 {'score': 0.05098661035299301,
  'sequence': 'Gaanụ mee ndị Izrel niile ka ha bụrụ ndị na- eso ụzọ m .',
  'token': 693,
  'token_str': ' Izrel'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask("Jehova họpụtara Mozis ka ọ bụrụ onye ndú ụmụ <mask>.") #= Izrel


[{'score': 0.46299779415130615,
  'sequence': 'Jehova họpụtara Mozis ka ọ bụrụ onye ndú ụmụ Izrel.',
  'token': 693,
  'token_str': ' Izrel'},
 {'score': 0.10745415091514587,
  'sequence': 'Jehova họpụtara Mozis ka ọ bụrụ onye ndú ụmụ mmadụ.',
  'token': 389,
  'token_str': ' mmadụ'},
 {'score': 0.10136278718709946,
  'sequence': 'Jehova họpụtara Mozis ka ọ bụrụ onye ndú ụmụ ya.',
  'token': 290,
  'token_str': ' ya'},
 {'score': 0.03240565210580826,
  'sequence': 'Jehova họpụtara Mozis ka ọ bụrụ onye ndú ụmụ ha.',
  'token': 297,
  'token_str': ' ha'},
 {'score': 0.01638958230614662,
  'sequence': 'Jehova họpụtara Mozis ka ọ bụrụ onye ndú ụmụ Livaị.',
  'token': 2441,
  'token_str': ' Livaị'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask("Ụmụakwụkwọ Chibok anọọla ụbọchị 2000 n’ aka <mask> Haram.") #= Boko


[{'score': 0.922619104385376,
  'sequence': 'Ụmụakwụkwọ Chibok anọọla ụbọchị 2000 n’ aka Boko Haram.',
  'token': 2608,
  'token_str': ' Boko'},
 {'score': 0.0041925013065338135,
  'sequence': 'Ụmụakwụkwọ Chibok anọọla ụbọchị 2000 n’ aka Man Haram.',
  'token': 2136,
  'token_str': ' Man'},
 {'score': 0.00371567509137094,
  'sequence': 'Ụmụakwụkwọ Chibok anọọla ụbọchị 2000 n’ aka Manchester Haram.',
  'token': 3417,
  'token_str': ' Manchester'},
 {'score': 0.0029909424483776093,
  'sequence': 'Ụmụakwụkwọ Chibok anọọla ụbọchị 2000 n’ aka Saụt Haram.',
  'token': 3099,
  'token_str': ' Saụt'},
 {'score': 0.0027692695148289204,
  'sequence': 'Ụmụakwụkwọ Chibok anọọla ụbọchị 2000 n’ aka soshal Haram.',
  'token': 3747,
  'token_str': ' soshal'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask("Nwunye Gọvanọ Ekiti steeti bụ Bisi Fayemi so na ndị na- akwado <mask> ọhụrụ a.") #= iwu


[{'score': 0.08747450262308121,
  'sequence': 'Nwunye Gọvanọ Ekiti steeti bụ Bisi Fayemi so na ndị na- akwado ọrụ ọhụrụ a.',
  'token': 468,
  'token_str': ' ọrụ'},
 {'score': 0.06200091168284416,
  'sequence': 'Nwunye Gọvanọ Ekiti steeti bụ Bisi Fayemi so na ndị na- akwado ụwa ọhụrụ a.',
  'token': 558,
  'token_str': ' ụwa'},
 {'score': 0.047154832631349564,
  'sequence': 'Nwunye Gọvanọ Ekiti steeti bụ Bisi Fayemi so na ndị na- akwado ọchịchị ọhụrụ a.',
  'token': 677,
  'token_str': ' ọchịchị'},
 {'score': 0.03148853778839111,
  'sequence': 'Nwunye Gọvanọ Ekiti steeti bụ Bisi Fayemi so na ndị na- akwado iwu ọhụrụ a.',
  'token': 670,
  'token_str': ' iwu'},
 {'score': 0.029196161776781082,
  'sequence': 'Nwunye Gọvanọ Ekiti steeti bụ Bisi Fayemi so na ndị na- akwado obodo ọhụrụ a.',
  'token': 565,
  'token_str': ' obodo'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask(" <mask> sị ka ehiwe ụlọikpe pụrụiche maka mpụ.") #= Buhari


[{'score': 0.293053537607193,
  'sequence': 'A sị ka ehiwe ụlọikpe pụrụiche maka mpụ.',
  'token': 37,
  'token_str': 'A'},
 {'score': 0.23877833783626556,
  'sequence': 'Ọ sị ka ehiwe ụlọikpe pụrụiche maka mpụ.',
  'token': 337,
  'token_str': 'Ọ'},
 {'score': 0.049465473741292953,
  'sequence': 'Ha sị ka ehiwe ụlọikpe pụrụiche maka mpụ.',
  'token': 515,
  'token_str': 'Ha'},
 {'score': 0.0299718976020813,
  'sequence': 'O sị ka ehiwe ụlọikpe pụrụiche maka mpụ.',
  'token': 51,
  'token_str': 'O'},
 {'score': 0.01546638086438179,
  'sequence': 'Anyị sị ka ehiwe ụlọikpe pụrụiche maka mpụ.',
  'token': 714,
  'token_str': 'Anyị'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask("Ala <mask>  ga- eweta ezi ọnọdụ nchekwa maka ndị chọrọ ịwebata ego n’ ọrụ ugbo.") #= Naịjirịa


[{'score': 0.2177654653787613,
  'sequence': 'Ala ahụ  ga- eweta ezi ọnọdụ nchekwa maka ndị chọrọ ịwebata ego n’ ọrụ ugbo.',
  'token': 307,
  'token_str': ' ahụ'},
 {'score': 0.03414909541606903,
  'sequence': 'Ala Nigeria  ga- eweta ezi ọnọdụ nchekwa maka ndị chọrọ ịwebata ego n’ ọrụ ugbo.',
  'token': 1264,
  'token_str': ' Nigeria'},
 {'score': 0.03212662413716316,
  'sequence': 'Ala Igbo  ga- eweta ezi ọnọdụ nchekwa maka ndị chọrọ ịwebata ego n’ ọrụ ugbo.',
  'token': 854,
  'token_str': ' Igbo'},
 {'score': 0.021992022171616554,
  'sequence': 'Ala,  ga- eweta ezi ọnọdụ nchekwa maka ndị chọrọ ịwebata ego n’ ọrụ ugbo.',
  'token': 16,
  'token_str': ','},
 {'score': 0.015841081738471985,
  'sequence': 'Ala a  ga- eweta ezi ọnọdụ nchekwa maka ndị chọrọ ịwebata ego n’ ọrụ ugbo.',
  'token': 266,
  'token_str': ' a'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask("Ọ bụ <mask>a ka a na- arịa .") #= mmadụ
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.032015010714530945,
  'sequence': 'Ọ bụ-a ka a na- arịa.',
  'token': 17,
  'token_str': '-'},
 {'score': 0.016286971047520638,
  'sequence': 'Ọ bụ nwaa ka a na- arịa.',
  'token': 420,
  'token_str': ' nwa'},
 {'score': 0.009335670620203018,
  'sequence': 'Ọ bụ Nwannaa ka a na- arịa.',
  'token': 1516,
  'token_str': ' Nwanna'},
 {'score': 0.00738544762134552,
  'sequence': 'Ọ bụ ịkwaa ka a na- arịa.',
  'token': 1479,
  'token_str': ' ịkwa'},
 {'score': 0.006971334107220173,
  'sequence': 'Ọ bụ adaa ka a na- arịa.',
  'token': 1050,
  'token_str': ' ada'}]

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#from google.colab import files
#files.download("/content/igbo_bert4.zip")

In [ ]:
shutil.move('/content/igbo_bert4','/content/gdrive/MyDrive/igbo_bert')

'/content/gdrive/MyDrive/igbo_bert/igbo_bert4'

In [ ]:
# shutil.make_archive("/content/igbo_bert4", 'zip', "igbo_bert4")

'/content/igbo_bert4.zip'

In [ ]:
# model_save_name = '/igbo_bert4.zip'
# path = F"/content/gdrive/My Drive/igbo_bert/{model_save_name}"
# torch.save(model.state_dict(), path)

In [ ]:
shutil.copy('/content/gdrive/MyDrive/igbo_bert/igbo_bert4','/content/sample_data')

IsADirectoryError: ignored